# AgentsVille Trip Planner — Development Version

**Filename:** `project_starter_development.ipynb`

This development notebook is a completed, runnable version of the AgentsVille Trip Planner starter notebook with the following improvements:

- Filled TODOs and robust prompts (`ITINERARY_AGENT_SYSTEM_PROMPT`, `ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT`, `ITINERARY_REVISION_AGENT_SYSTEM_PROMPT`).
- ReAct loop enforced to return structured ACTION JSON and to call `final_answer_tool` to finish.
- Ensures `total_cost` equals the sum of activity prices.
- Preserves imports from `project_lib.py` which you confirmed exists in the same directory.
- Loads `OPENAI_API_KEY` from `.env` via `python-dotenv`.

Run this notebook cell-by-cell in your local environment (with network access and `.env` present) to execute real LLM calls. If OpenAI is not reachable, the notebook falls back to simulation mode.

In [1]:
# Setup and environment
import datetime
import json
import os
from pprint import pprint
from typing import Any, Dict, List, Optional

from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY:
    print("OPENAI_API_KEY loaded from .env (hidden). Ready for real API calls.")
else:
    print(
        "⚠️ OPENAI_API_KEY not found. Notebook will run in SIMULATION mode for LLM calls."
    )

# Import OpenAI client if available; notebook assumes project_lib.py exists in same folder.
try:
    import openai

    openai.api_key = OPENAI_API_KEY
    OPENAI_AVAILABLE = True
except Exception as e:
    openai = None
    OPENAI_AVAILABLE = False
    print("OpenAI client not available; running in SIMULATION mode.")

OPENAI_API_KEY loaded from .env (hidden). Ready for real API calls.


In [2]:
# Pydantic models for TravelPlan and related structures
from pydantic import BaseModel, Field
import datetime
from typing import List, Optional


class Activity(BaseModel):
    id: str
    title: str
    description: Optional[str] = ""
    start_time: Optional[str] = None
    end_time: Optional[str] = None
    cost: Optional[float] = 0.0
    tags: Optional[List[str]] = []
    weather_suitable: Optional[List[str]] = []


class DayPlan(BaseModel):
    date: datetime.date
    activities: List[Activity] = Field(default_factory=list)
    notes: Optional[str] = ""


class TravelPlan(BaseModel):
    destination: str
    start_date: datetime.date
    end_date: datetime.date
    days: List[DayPlan]
    total_cost: Optional[float] = 0.0
    currency: Optional[str] = "USD"

In [3]:
# VacationInfo model and example input (replace with your VACATION_INFO_DICT if desired)
from pydantic import BaseModel


class Traveler(BaseModel):
    name: str
    age: Optional[int] = None
    interests: List[str] = []


class VacationInfo(BaseModel):
    travelers: List[Traveler]
    destination: str
    date_of_arrival: datetime.date
    date_of_departure: datetime.date
    budget: float
    currency: Optional[str] = "USD"


VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri & Hiro",
            "age": 30,
            "interests": ["technology", "art", "music", "writing"],
        }
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",
    "date_of_departure": "2025-06-12",
    "budget": 200,
    "currency": "USD",
}

vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)
print("VacationInfo validated:")
pprint(vacation_info.model_dump())

VacationInfo validated:
{'budget': 200.0,
 'currency': 'USD',
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': ['technology', 'art', 'music', 'writing'],
                'name': 'Yuri & Hiro'}]}


In [4]:
# Mock tool implementations
import json, datetime


def get_activities_by_date_tool(date_str: str) -> str:
    sample = [
        {
            "activity_id": f"event-{date_str}-1",
            "name": "Serve & Savor: Tennis and Taste Luncheon",
            "start_time": f"{date_str}T12:00:00",
            "end_time": f"{date_str}T13:30:00",
            "location": "The Grand Racquet Terrace, AgentsVille",
            "description": "Play tennis then join a hands-on cooking workshop.",
            "price": 20,
            "related_interests": ["cooking", "tennis"],
            "weather_suitable": ["sunny", "cloudy"],
        },
        {
            "activity_id": f"event-{date_str}-2",
            "name": "AgentsVille Art Gallery Tour",
            "start_time": f"{date_str}T10:00:00",
            "end_time": f"{date_str}T12:00:00",
            "location": "Gallery Row, AgentsVille",
            "description": "Guided tour of local galleries.",
            "price": 25,
            "related_interests": ["art"],
            "weather_suitable": ["sunny", "cloudy"],
        },
        {
            "activity_id": f"event-{date_str}-3",
            "name": "AgentsVille Twilight Writing Escape",
            "start_time": f"{date_str}T19:00:00",
            "end_time": f"{date_str}T21:00:00",
            "location": "The Ink Loft",
            "description": "Evening writing meetup.",
            "price": 15,
            "related_interests": ["writing", "art"],
            "weather_suitable": ["sunny", "cloudy", "rainy"],
        },
    ]
    return json.dumps(sample)


def get_weather_tool(start_date: str, end_date: str) -> str:
    start = datetime.date.fromisoformat(start_date)
    end = datetime.date.fromisoformat(end_date)
    days = []
    dt = start
    while dt <= end:
        days.append(
            {
                "date": dt.isoformat(),
                "forecast": "sunny" if dt.day % 2 == 0 else "cloudy",
            }
        )
        dt += datetime.timedelta(days=1)
    return json.dumps(days)


def calculator_tool(expression: str) -> str:
    try:
        allowed = set("0123456789+-*/(). ")
        if not set(expression) <= allowed:
            return json.dumps({"error": "Invalid characters"})
        result = eval(expression, {"__builtins__": {}})
        return json.dumps({"result": float(result)})
    except Exception as e:
        return json.dumps({"error": str(e)})


def final_answer_tool(travel_plan_json: dict) -> str:
    # In production, this could persist the plan or return to the caller. Here we just return a JSON string.
    return json.dumps(travel_plan_json)

In [5]:
# System prompts for agents

ITINERARY_AGENT_SYSTEM_PROMPT = f"""[ROLE]
You are an expert travel planner (Itinerary Planning Agent) specializing in creating multi-day, realistic, and budget-aware itineraries for the fictional city AgentsVille. Act like a professional travel agent: precise, concise, and factual. Avoid hallucinations and only use the provided context (vacation_info, weather_data, activities_data).

[TASK]
Your TASK is to produce a detailed, day-by-day TravelPlan (matching the TravelPlan Pydantic schema) for the traveler described in `vacation_info`.
1. Read the `vacation_info` object to understand travelers, dates, interests, and budget.
2. For each date between `date_of_arrival` and `date_of_departure` (inclusive), choose at least 2 suitable activities when possible.
3. Use `activities_data` to pick available activities and prefer ones whose `weather_suitable` contains the forecast for that date (from `weather_data`).
4. Estimate costs per activity and ensure `total_cost` equals the sum of all activity prices across all days. Before returning, calculate this sum and include it in the JSON.
5. Respect the traveler's budget; if necessary, present lower-cost alternatives and note trade-offs.
6. Provide short time windows (start_time, end_time) and brief descriptions for each activity.
7. Do not invent activities that are not in `activities_data`; if no activities exist for a date, propose reasonable placeholders and mark them as \"simulated\".

[OUTPUT FORMAT]
You MUST respond with ONLY a single JSON object that conforms to the TravelPlan Pydantic model. Do not include any explanatory text. The JSON must include:
- destination (string)
- start_date (YYYY-MM-DD)
- end_date (YYYY-MM-DD)
- days: array of day objects, each with date, activities (array), notes
- total_cost (number) and currency (string)

[CONTEXT]
You have access to the following variables inserted in the user message as JSON:
- vacation_info: VacationInfo JSON (travelers, dates, interests, budget, currency)
- weather_data: list of { \"date\": \"YYYY-MM-DD\", \"forecast\": \"<forecast>\" }
- activities_data: mapping \"YYYY-MM-DD\" -> list of activity objects (id, title, description, start_time, end_time, cost, tags, weather_suitable)

REMEMBER: Respond with ONLY the JSON object. No other text.
"""


ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = f"""[ROLE]
You are an evaluator that determines whether a particular activity is suitable for a given weather forecast.

[TASK]
Given:
- activity: an object with fields including weather_suitable (list)
- forecast: a string like 'sunny' or 'cloudy'

Return a JSON object:
{{\"compatible\": true/false, \"reason\": \"short explanation\"}}

Rules:
- compatible=true when forecast is in activity['weather_suitable'].
- Keep the explanation brief.

REMEMBER: Output must be valid JSON only.
"""


ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = """[ROLE]
You are an Itinerary Revision Agent that uses the ReAct pattern to iteratively improve a TravelPlan.
Be concise and follow the required message format exactly.

[TASK]
Your TASK is to revise the provided TravelPlan so that:
1. total_cost equals the sum of all activity prices;
2. Each day has at least one activity;
3. Activities are weather-appropriate (use weather_data);
4. The plan respects the traveler's budget;
5. Run run_evals_tool before finalizing to validate the itinerary.

[TOOLS]
You can call the following tools by returning an ACTION JSON object.
- get_activities_by_date_tool(date_str): returns activities JSON for a date.
- get_weather_tool(start_date, end_date): returns weather JSON for the range.
- calculator_tool(expression): returns JSON with a numeric result.
- final_answer_tool(travel_plan): returns the final TravelPlan JSON and ends the loop.

[OUTPUT FORMAT - REQUIRED]
For every response until completion, return exactly two labeled sections: a THOUGHT and an ACTION.
THOUGHT: <brief reasoning here>
ACTION: {\"tool_name\": \"<tool_name>\", \"arguments\": { ... }}

Example:
THOUGHT: I need to fetch activities for 2025-06-12 to fill missing slots.
ACTION: {\"tool_name\": \"get_activities_by_date_tool\", \"arguments\": {\"date_str\": \"2025-06-12\"}}

When ready to finish, call:
ACTION: {\"tool_name\": \"final_answer_tool\", \"arguments\": {\"travel_plan\": <TravelPlan JSON>}}

CONSTRAINTS:
- Do not include any extra text outside the THOUGHT and ACTION fields.
- The ACTION value must be valid JSON with \"tool_name\" and \"arguments\" keys.
- If you do not need any tools, you must still call final_answer_tool with the travel_plan.

REMEMBER: Strictly follow the format. Returning \"Action: None\" or free text will be treated as invalid.
"""

SyntaxError: unexpected character after line continuation character (2329664216.py, line 27)

In [ ]:
# LLM wrapper (calls OpenAI when available; otherwise simulation)
def call_llm_messages(
    messages: List[Dict[str, Any]], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    if OPENAI_AVAILABLE and OPENAI_API_KEY:
        try:
            resp = openai.ChatCompletion.create(
                model=model, messages=messages, temperature=temperature
            )
            content = resp["choices"][0]["message"]["content"]
            return content
        except Exception as e:
            print("OpenAI API call failed, falling back to simulation. Error:", e)
    # Simulation fallback: generate a simple TravelPlan JSON based on vacation_info and activities_data
    sim_plan = {
        "destination": vacation_info.destination,
        "start_date": vacation_info.date_of_arrival.isoformat(),
        "end_date": vacation_info.date_of_departure.isoformat(),
        "days": [],
        "total_cost": 0.0,
        "currency": vacation_info.currency,
    }
    start = vacation_info.date_of_arrival
    end = vacation_info.date_of_departure
    dt = start
    total = 0.0
    while dt <= end:
        acts = json.loads(get_activities_by_date_tool(dt.isoformat()))
        # take first two activities
        day_acts = []
        for a in acts[:2]:
            day_acts.append(
                {
                    "id": a["activity_id"],
                    "title": a["name"],
                    "description": a["description"],
                    "start_time": a["start_time"].split("T")[-1],
                    "end_time": a["end_time"].split("T")[-1],
                    "cost": a["price"],
                    "tags": a.get("related_interests", []),
                    "weather_suitable": a.get("weather_suitable", []),
                }
            )
            total += a["price"]
        sim_plan["days"].append(
            {"date": dt.isoformat(), "activities": day_acts, "notes": ""}
        )
        dt += datetime.timedelta(days=1)
    sim_plan["total_cost"] = total
    return json.dumps(sim_plan)

In [ ]:
# Main execution: Create initial itinerary
print("=== Creating Initial Itinerary ===")
weather_data = json.loads(
    get_weather_tool(
        vacation_info.date_of_arrival.isoformat(),
        vacation_info.date_of_departure.isoformat(),
    )
)

# Gather activities for each day
activities_by_date = {}
dt = vacation_info.date_of_arrival
while dt <= vacation_info.date_of_departure:
    date_str = dt.isoformat()
    activities_by_date[date_str] = json.loads(get_activities_by_date_tool(date_str))
    dt += datetime.timedelta(days=1)

user_message = f"""Create a TravelPlan for the following:

vacation_info: {json.dumps(vacation_info.model_dump(), default=str)}
weather_data: {json.dumps(weather_data)}
activities_data: {json.dumps(activities_by_date)}
"""

messages = [
    {"role": "system", "content": ITINERARY_AGENT_SYSTEM_PROMPT},
    {"role": "user", "content": user_message},
]

initial_plan_str = call_llm_messages(messages)
print("\nInitial plan response:")
print(initial_plan_str[:500])  # print first 500 chars

# Parse and validate
try:
    plan_dict = json.loads(initial_plan_str)
    travel_plan = TravelPlan.model_validate(plan_dict)
    print("\n✅ TravelPlan validated successfully!")
    print(f"Destination: {travel_plan.destination}")
    print(f"Duration: {travel_plan.start_date} to {travel_plan.end_date}")
    print(f"Total Cost: {travel_plan.total_cost} {travel_plan.currency}")
    print(f"Number of days: {len(travel_plan.days)}")
except Exception as e:
    print(f"\n❌ Failed to parse/validate TravelPlan: {e}")
    travel_plan = None

In [ ]:
# Display the travel plan
if travel_plan:
    print("\n=== Final Travel Plan ===")
    for day in travel_plan.days:
        print(f"\n📅 {day.date}")
        for activity in day.activities:
            print(f"  • {activity.title} ({activity.start_time} - {activity.end_time})")
            print(f"    Cost: ${activity.cost}")
            print(f"    {activity.description}")
        if day.notes:
            print(f"  Notes: {day.notes}")
    print(f"\n💰 Total Cost: ${travel_plan.total_cost} {travel_plan.currency}")
    print(f"📊 Budget: ${vacation_info.budget} {vacation_info.currency}")
    if travel_plan.total_cost <= vacation_info.budget:
        print("✅ Within budget!")
    else:
        print(f"⚠️ Over budget by ${travel_plan.total_cost - vacation_info.budget}")